In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from dataset import make_dataset
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

<h1> 0. Final Results

In [2]:
MSE_ans = {'Model': ['2.1', '2.2', '2.3', '2.4', '2.5', '2.6'],
        'MSE': [225.02911302400184,224.98720382082988,230.86344632847118,241.42927938269833,227.5116066048393,228.2871445526927]}
MSE_table = pd.DataFrame(MSE_ans)
MSE_table = MSE_table.sort_values(by='MSE')
MSE_table.index = range(len(MSE_table))
MSE_table

,Model,MSE
0,2.2,224.987204
1,2.1,225.029113
2,2.5,227.511607
3,2.6,228.287145
4,2.3,230.863446
5,2.4,241.429279


Based on the result, the best model is Model 2.2 
- X: Remove productID
- Y: sales
- Parameters: {'ccp_alpha': 0.0, 'max_depth': 4, 'min_samples_leaf': 4, 'min_samples_split': 2}

<h1> 1. Data Preprocess

<h2> 1.1 Import Data

In [3]:
ss, not_scaled_df, scaled_df, data_train = make_dataset()
not_scaled_df

/Users/cyl/opt/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,productID_0,productID_1,productID_2,productID_3,productID_6,productID_7,productID_8,productID_9,productID_10,productID_13,...,ma14SalesVolume,meanAge,gender,meanEducation,maritalStatus,plus,meanPurchasePower,meanUserLevel,meanCityLevel,sales
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.500000,28.000000,0.400000,3.000000,0.800000,0.000000,2.200000,2.666667,2.250000,7
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,21.142857,32.444444,0.222222,3.333333,0.555556,0.181818,2.333333,3.000000,1.777778,14
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,36.214286,32.954545,0.431818,3.441860,0.727273,0.555556,2.181818,3.133333,2.000000,60
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.928571,40.000000,0.500000,3.500000,1.000000,0.500000,2.500000,3.000000,1.000000,2
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,18.214286,33.333333,0.083333,2.900000,0.800000,0.000000,2.500000,2.142857,2.900000,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2206,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.214286,30.000000,0.200000,3.250000,0.800000,0.428571,2.400000,2.571429,2.250000,8
2207,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,22.714286,31.294118,0.470588,2.937500,0.625000,0.294118,2.250000,2.588235,2.176471,18
2208,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.000000,25.000000,0.250000,3.000000,0.750000,0.000000,2.500000,1.600000,2.000000,7
2209,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.642857,28.750000,0.250000,2.571429,0.571429,0.250000,2.285714,2.125000,2.125000,10


In [4]:
scaled_df

,productID_0,productID_1,productID_2,productID_3,productID_6,productID_7,productID_8,productID_9,productID_10,productID_13,...,ma14SalesVolume,meanAge,gender,meanEducation,maritalStatus,plus,meanPurchasePower,meanUserLevel,meanCityLevel,sales
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.948957,-0.822878,0.524839,-0.176485,1.066933,-1.260993,-0.205651,-0.229769,0.157289,7
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.239057,0.123052,-0.264700,0.885324,0.092044,-0.370702,0.346301,0.205788,-0.711543,14
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.253920,0.231618,0.666149,1.231029,0.776884,1.459339,-0.280918,0.380011,-0.302681,60
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.583414,1.731132,0.968955,1.416228,1.864569,1.187306,1.036241,0.205788,-2.142560,2
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.041857,0.312237,-0.881527,-0.495027,1.066933,-1.260993,1.036241,-0.914216,1.353210,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2206,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.631512,-0.397210,-0.363392,0.619872,1.066933,0.837549,0.622277,-0.354214,0.157289,8
2207,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.344872,-0.121777,0.838332,-0.375574,0.369001,0.179183,0.001331,-0.332253,0.022004,18
2208,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.780615,-1.461380,-0.141334,-0.176485,0.867524,-1.260993,1.036241,-1.623552,-0.302681,7
2209,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.669990,-0.663252,-0.141334,-1.541667,0.155349,-0.036844,0.149175,-0.937550,-0.072696,10


<h2> 1.2 Filter Data

In [13]:
data = scaled_df[scaled_df['sales'] <= 200]
x_train=data.drop(['sales'],axis=1)
x_train

,productID_0,productID_1,productID_2,productID_3,productID_6,productID_7,productID_8,productID_9,productID_10,productID_13,...,avgFinalUnitPrice,ma14SalesVolume,meanAge,gender,meanEducation,maritalStatus,plus,meanPurchasePower,meanUserLevel,meanCityLevel
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.176703,-0.948957,-0.822878,0.524839,-0.176485,1.066933,-1.260993,-0.205651,-0.229769,0.157289
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.795620,0.239057,0.123052,-0.264700,0.885324,0.092044,-0.370702,0.346301,0.205788,-0.711543
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.937825,1.253920,0.231618,0.666149,1.231029,0.776884,1.459339,-0.280918,0.380011,-0.302681
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.537158,-0.583414,1.731132,0.968955,1.416228,1.864569,1.187306,1.036241,0.205788,-2.142560
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.470020,0.041857,0.312237,-0.881527,-0.495027,1.066933,-1.260993,1.036241,-0.914216,1.353210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2206,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.137396,-0.631512,-0.397210,-0.363392,0.619872,1.066933,0.837549,0.622277,-0.354214,0.157289
2207,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.292718,0.344872,-0.121777,0.838332,-0.375574,0.369001,0.179183,0.001331,-0.332253,0.022004
2208,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.407518,-0.780615,-1.461380,-0.141334,-0.176485,0.867524,-1.260993,1.036241,-1.623552,-0.302681
2209,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.071468,-0.669990,-0.663252,-0.141334,-1.541667,0.155349,-0.036844,0.149175,-0.937550,-0.072696


In [14]:
y_train=data['sales']
y_train

0        7
1       14
2       60
3        2
4       15
        ..
2206     8
2207    18
2208     7
2209    10
2210     3
Name: sales, Length: 2204, dtype: int64

<h1> 2. Decision Tree

<h2> 2.1 Grid Search for DT based on Full dataset

In [15]:
# Creating the model
model = DecisionTreeRegressor(random_state=24)

# Dataset used:
X=x_train
y=y_train

# Parameter grid to vary
param_grid = {'max_depth': [2, 4, 6, 8, 10],
              'min_samples_split': [2, 4, 5, 6, 8],
              'min_samples_leaf': [1, 2, 4, 6],
              'ccp_alpha': [0.0, 0.001, 0.01, 0.05, 0.1, 1.0]}

# Creating the object to carry out GridSearchCV, with 10-Fold Cross Validation
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', n_jobs=-1, cv=10, return_train_score=True)
grid_search.fit(X,y)

# Output the best parameters and its corresponding test MSE
print(f"The best parameter for the current model is {grid_search.best_params_}")
print(f"The corresponding test MSE is {np.abs(grid_search.cv_results_['mean_test_score'][grid_search.best_index_])}")

The best parameter for the current model is {'ccp_alpha': 0.0, 'max_depth': 4, 'min_samples_leaf': 4, 'min_samples_split': 2}
The corresponding test MSE is 225.02911302400184


<h2> 2.2 Grid Search for DT based on dataset with removed productID

In [17]:
# Get a new DataFrame without the productID columns
x_train_noproductID = x_train.drop(columns=[col for col in x_train.columns if col.startswith('productID_')], inplace=False)

In [18]:
# Creating the model
model = DecisionTreeRegressor(random_state=24)

# Dataset used:
X=x_train_noproductID
y=y_train

# Parameter grid to vary
param_grid = {'max_depth': [2, 4, 6, 8, 10],
              'min_samples_split': [2, 4, 5, 6, 8],
              'min_samples_leaf': [1, 2, 4, 6],
              'ccp_alpha': [0.0, 0.001, 0.01, 0.05, 0.1, 1.0]}

# Creating the object to carry out GridSearchCV, with 10-Fold Cross Validation
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', n_jobs=-1, cv=10, return_train_score=True)
grid_search.fit(X,y)

# Output the best parameters and its corresponding test MSE
print(f"The best parameter for the current model is {grid_search.best_params_}")
print(f"The corresponding test MSE is {np.abs(grid_search.cv_results_['mean_test_score'][grid_search.best_index_])}")

The best parameter for the current model is {'ccp_alpha': 0.0, 'max_depth': 4, 'min_samples_leaf': 4, 'min_samples_split': 2}
The corresponding test MSE is 224.98720382082988


<h2> 2.3 Grid Search for DT based on dataset with removed brandID

In [19]:
# Get a new DataFrame without the brandID columns
x_train_nobrandID = x_train.drop(columns=[col for col in x_train.columns if col.startswith('brandID_')], inplace=False)

In [20]:
# Creating the model
model = DecisionTreeRegressor(random_state=24)

# Dataset used:
X=x_train_nobrandID
y=y_train

# Parameter grid to vary
param_grid = {'max_depth': [2, 4, 6, 8, 10],
              'min_samples_split': [2, 4, 5, 6, 8],
              'min_samples_leaf': [1, 2, 4, 6],
              'ccp_alpha': [0.0, 0.001, 0.01, 0.05, 0.1, 1.0]}

# Creating the object to carry out GridSearchCV, with 10-Fold Cross Validation
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', n_jobs=-1, cv=10, return_train_score=True)
grid_search.fit(X,y)

# Output the best parameters and its corresponding test MSE
print(f"The best parameter for the current model is {grid_search.best_params_}")
print(f"The corresponding test MSE is {np.abs(grid_search.cv_results_['mean_test_score'][grid_search.best_index_])}")

The best parameter for the current model is {'ccp_alpha': 0.0, 'max_depth': 4, 'min_samples_leaf': 4, 'min_samples_split': 2}
The corresponding test MSE is 230.86344632847118


<h2> 2.4 Grid Search for DT based on dataset with removed avgFinalUnitPrice

In [21]:
# Get a new DataFrame without the avgFinalUnitPrice columns
x_train_noavgFinalUnitPrice = x_train.drop(['avgFinalUnitPrice'],axis=1)

In [22]:
# Creating the model
model = DecisionTreeRegressor(random_state=24)

# Dataset used:
X=x_train_noavgFinalUnitPrice
y=y_train

# Parameter grid to vary
param_grid = {'max_depth': [2, 4, 6, 8, 10],
              'min_samples_split': [2, 4, 5, 6, 8],
              'min_samples_leaf': [1, 2, 4, 6],
              'ccp_alpha': [0.0, 0.001, 0.01, 0.05, 0.1, 1.0]}

# Creating the object to carry out GridSearchCV, with 10-Fold Cross Validation
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', n_jobs=-1, cv=10, return_train_score=True)
grid_search.fit(X,y)

# Output the best parameters and its corresponding test MSE
print(f"The best parameter for the current model is {grid_search.best_params_}")
print(f"The corresponding test MSE is {np.abs(grid_search.cv_results_['mean_test_score'][grid_search.best_index_])}")

The best parameter for the current model is {'ccp_alpha': 0.1, 'max_depth': 6, 'min_samples_leaf': 6, 'min_samples_split': 2}
The corresponding test MSE is 241.42927938269833


<h2> 2.5 Grid Search for DT based on dataset with removed avgOriginalUnitPrice

In [23]:
# Get a new DataFrame without the avgOriginalUnitPrice columns
x_train_noavgOriginalUnitPrice = x_train.drop(['avgOriginalUnitPrice'],axis=1)

In [24]:
# Creating the model
model = DecisionTreeRegressor(random_state=24)

# Dataset used:
X=x_train_noavgOriginalUnitPrice
y=y_train

# Parameter grid to vary
param_grid = {'max_depth': [2, 4, 6, 8, 10],
              'min_samples_split': [2, 4, 5, 6, 8],
              'min_samples_leaf': [1, 2, 4, 6],
              'ccp_alpha': [0.0, 0.001, 0.01, 0.05, 0.1, 1.0]}

# Creating the object to carry out GridSearchCV, with 10-Fold Cross Validation
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', n_jobs=-1, cv=10, return_train_score=True)
grid_search.fit(X,y)

# Output the best parameters and its corresponding test MSE
print(f"The best parameter for the current model is {grid_search.best_params_}")
print(f"The corresponding test MSE is {np.abs(grid_search.cv_results_['mean_test_score'][grid_search.best_index_])}")

The best parameter for the current model is {'ccp_alpha': 0.0, 'max_depth': 4, 'min_samples_leaf': 4, 'min_samples_split': 2}
The corresponding test MSE is 227.5116066048393


<h2> 2.6 Grid Search for DT based on dataset with interacting term of avgOriginalUnitPrice and avgFinalUnitPrice

In [28]:
#add interacting term of avgOriginalUnitPrice and avgFinalUnitPrice
Data=not_scaled_df.copy()
Data= Data[Data['sales'] <= 200]
Data['avgOriginalUnitPrice*avgFinalUnitPrice'] = Data['avgOriginalUnitPrice'] * Data['avgFinalUnitPrice']
x_avgOriginalUnitPrice_int_avgFinalUnitPrice=Data.drop('sales', axis=1)
ss=StandardScaler(with_mean=False)
x_train_avgOriginalUnitPrice_int_avgFinalUnitPrice= ss.fit_transform(x_avgOriginalUnitPrice_int_avgFinalUnitPrice)
x_train_avgOriginalUnitPrice_int_avgFinalUnitPrice=pd.DataFrame(x_train_avgOriginalUnitPrice_int_avgFinalUnitPrice,columns=x_avgOriginalUnitPrice_int_avgFinalUnitPrice.columns)

In [29]:
# Creating the model
model = DecisionTreeRegressor(random_state=24)

# Dataset used:
X=x_train_avgOriginalUnitPrice_int_avgFinalUnitPrice
y=y_train

# Parameter grid to vary
param_grid = {'max_depth': [2, 4, 6, 8, 10],
              'min_samples_split': [2, 4, 5, 6, 8],
              'min_samples_leaf': [1, 2, 4, 6],
              'ccp_alpha': [0.0, 0.001, 0.01, 0.05, 0.1, 1.0]}

# Creating the object to carry out GridSearchCV, with 10-Fold Cross Validation
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', n_jobs=-1, cv=10, return_train_score=True)
grid_search.fit(X,y)

# Output the best parameters and its corresponding test MSE
print(f"The best parameter for the current model is {grid_search.best_params_}")
print(f"The corresponding test MSE is {np.abs(grid_search.cv_results_['mean_test_score'][grid_search.best_index_])}")

The best parameter for the current model is {'ccp_alpha': 0.0, 'max_depth': 4, 'min_samples_leaf': 4, 'min_samples_split': 2}
The corresponding test MSE is 228.2871445526927
